In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sympy.combinatorics import Permutation


class RubiksCube():
    """
    Realization of Rubik's Cube in 2D with the capability to simulate moves and solve arbitrary states.

    Parameters:
        - state: str
            A string of size 54 representing the mesh form of the cube using characters only from {'y', 'b', 'r', 'g', 'o', 'w'}.
            Default value is "yyyyyyyyybbbbbbbbbrrrrrrrrrgggggggggooooooooowwwwwwwww", which is the default state of the cube. 
    """

    colors = {
        'y': np.uint8([255, 213,   0]),     # yellow
        'b': np.uint8([  0,  70, 173]),     # blue
        'r': np.uint8([183,  18,  52]),     # red
        'g': np.uint8([  0, 155,  72]),     # green
        'o': np.uint8([255,  88,   0]),     # orange
        'w': np.uint8([255, 255, 255]),     # white
        'q': np.uint8([199, 208, 218])      # quarzo - background
    }

    def __init__(self, state = "yyyyyyyyybbbbbbbbbrrrrrrrrrgggggggggooooooooowwwwwwwww"):
        assert isinstance(state, str) and len(state) == 54, "Argument should be a string of size 54"
        assert set(list(state)).issubset({'y', 'b', 'r', 'g', 'o', 'w'}), "All characters should be from {'y', 'b', 'r', 'g', 'o', 'w'}"

        self.state = np.array(list(state))

    def string_rep(self):
        """
        Return string representation of current state.
        """

        return "".join(i for i in list(self.state))
    
    def show(self):
        """
        Display current state of the cube on a 2D colored mesh.
        """

        # creating colored mesh of current state
        faces = self.state.reshape(6, 3, 3)     # an array of size 6 containing current state of faces in matrices of size 3x3
        block_q = np.full((3, 3), 'q')          # background cells
        matrix_form = np.block([[block_q,  faces[0], block_q,  block_q ],
                                [faces[1], faces[2], faces[3], faces[4]],
                                [block_q,  faces[5], block_q,  block_q ]])  # a matrix of size 9x12 built from 3x3 blocks
        mesh_data = [[self.colors[i] for i in row] for row in matrix_form]

        # displaying the colored mesh
        fig, ax = plt.subplots()
        ax.invert_yaxis()   # inverting axis y, so first line will be on top
        ax.set_xticks([])   # no ticks and labels on axis x
        ax.set_yticks([])   # no ticks and labels on axis y
        ax.pcolormesh(mesh_data, edgecolors = 'k', linewidth = 0.5)


    layer_rotations = {
        'F': Permutation(53)(6,17,47,27)(7,14,46,30)(8,11,45,33)(18,24,26,20)(19,21,25,23),     # front - the middle one on the image
        'B': Permutation(53)(0,29,53,15)(1,32,52,12)(2,35,51,9)(36,42,44,38)(37,39,43,41),      # back
        'U': Permutation(53)(0,6,8,2)(1,3,7,5)(9,18,27,36)(10,19,28,37)(11,20,29,38),           # up
        'D': Permutation(53)(15,42,33,24)(16,43,34,25)(17,44,35,26)(45,51,53,47)(46,48,52,50),  # down
        'L': Permutation(53)(0,44,45,18)(3,41,48,21)(6,38,51,24)(9,15,17,11)(10,12,16,14),      # left
        'R': Permutation(53)(2,20,47,42)(5,23,50,39)(8,26,53,36)(27,33,35,29)(28,30,34,32),     # right
        'M': Permutation(53)(1,43,46,19)(4,40,49,22)(7,37,52,25),                               # middle
        'E': Permutation(53)(12,39,30,21)(13,40,31,22)(14,41,32,23),                            # equator
        'S': Permutation(53)(3,16,50,28)(4,13,49,31)(5,10,48,34)                                # standing
    }

    cube_rotations = {
        'X': layer_rotations['R'] * layer_rotations['M']**(-1) * layer_rotations['L']**(-1),    # rotate cube as R
        'Y': layer_rotations['U'] * layer_rotations['E']**(-1) * layer_rotations['D']**(-1),    # rotate cube as U
        'Z': layer_rotations['F'] * layer_rotations['S']       * layer_rotations['B']**(-1)     # rotate cube as F
    }

    base_moves    = {**layer_rotations, **cube_rotations}                           # clockwise rotations
    inverse_moves = {key + "'": value**(-1) for key, value in base_moves.items()}   # counterclockwise rotations, indicated with an "'"
    double_moves  = {key + "2": value**2 for key, value in base_moves.items()}      # double rotations, indicated with a "2"

    all_moves = {**base_moves, **inverse_moves, **double_moves}                     # all possible moves 

    def move(self, move):
        """
        Take a simple move.

        Parameters:
            - move: str
                Valid moves are 'F', 'B', 'U', 'D', 'L', 'R', 'M', 'E', 'S', 'X', 'Y', 'Z' optionally followed by a "'" or a "2".
        """

        assert move in self.all_moves.keys(), f'Argument should be from {list(self.base_moves.keys())} with optionally followed by an "\'" or a "2".'

        permutation = self.all_moves[move]
        self.state = np.array(permutation(self.state))

    def algorithm(self, algorithm = "", iterations = 1):
        """
        Execute an algorithm on the cube.

        Parameters:
            - algorithm: str
                A sequence of valid moves separated white whitespaces. 
                Valid moves are 'F', 'B', 'U', 'D', 'L', 'R', 'M', 'E', 'S', 'X', 'Y', 'Z' with optionally followed by an "'" or a "2".
            - iterations: int
                The number of times the algorithm needs to be executed.
        """

        assert isinstance(iterations, int), f'Parameter \'iterations\' should be an integer.'
        assert iterations >= 0, f'Parameter \'iterations\' should be a non-negative number.'

        sequence = algorithm.split()
        for i in range(iterations):
            for move in sequence:
                self.move(move)

    def scramble(self, moves = 50):
        """
        Scramble the cube with random moves.

        Parameters:
            - moves: int
                The number of random moves.
        """

        assert isinstance(moves, int), f'Parameter \'moves\' should be an integer.'
        assert moves >= 1, f'Parameter \'moves\' should be a positive number.'

        for i in range(moves):
            move = random.choice(list(self.layer_rotations.keys()))
            self.move(move)

    def reset(self):
        """
        Reset the cube to default state.
        """
        self.__init__()

    def colors_match(self, area_list) -> bool:
        """
        Check if colors match in the given areas.
        Return True if each area has a unique color, otherwise return False.

        Parameters:
            - area_list: list[list[int]]
                The list of the areas to check. Each sublist represents an area and should contain the corresponding positions.
                An area is an arbitrary subset of all positions, i.e. not necessarily connected.
        """

        match = True
        for area in area_list:
            colors_in_area = [self.state[pos] for pos in area]
            match = match and len(set(colors_in_area)) <= 1
        return match

    def compress_algorithm(self, algorithm) -> str:
        """
        Compress algorithm, i.e. substitute consecutive moves with a single move if possible.
        The result will be also cleaned from neighboring inverse moves.

        Parameters:
            - algorithm: str
                The algorithm to compress.
        """

        moves:list = algorithm.split()

        # Substitute consecutive moves with a single move if possible
        # It can be the empty move as well 
        i = 0
        while True:
            if i >= len(moves) - 1:
                break
            substituter = self.combine(moves[i], moves[i+1])
            if substituter:
                moves[i] = substituter
                del moves[i+1]
                if substituter != " ":  # if the substituter is the empty move, then we should increase i, otherwise not
                    continue
            i += 1

        # Remove empty moves
        moves = [move for move in moves if move != " "]

        return " ".join(moves)

    def combine(self, move1, move2) -> str:
        """
        Combine two moves. Return an equivalent single move if exists, otherwise return None. Return value can be the empty move as well.

        Parameters:
            - move1: str
                The first move to combine.
            - move2: str
                The second move to combine.
        """

        move1 = "".join(move1.split())  # remove whitespaces
        move2 = "".join(move2.split())

        perm1:Permutation = self.all_moves[move1]
        perm2:Permutation = self.all_moves[move2]
        combination = perm1 * perm2
        
        if combination.is_identity:
            return " "
        for key, value in self.all_moves.items():
            if combination == value:
                return key


    """

    Solver

    """

    def solve(self, method = "CFOP"):
        """
        Solve the cube using the given method.

        Parameters:
            - method: str
                The name of a method to solve the cube. Currently 'CFOP' is the only option.
        """

        if method == "CFOP":
            print("Solve via CFOP method \n")
            print(f"Input: {self.string_rep()} \n")

            cross_moves = self.CFOP_cross()
            cross_solution = self.compress_algorithm(" ".join(cross_moves))
            print(f"Cross: {cross_solution}")

            F2L_moves = self.CFOP_F2L()
            F2L_solution = self.compress_algorithm(" ".join(F2L_moves))
            print(f" F2L : {F2L_solution}")

            OLL_moves = self.CFOP_OLL()
            OLL_solution = self.compress_algorithm(" ".join(OLL_moves))
            print(f" OLL : {OLL_solution}")

            PLL_moves = self.CFOP_PLL()
            PLL_solution = self.compress_algorithm(" ".join(PLL_moves))
            print(f" PLL : {PLL_solution} \n")

            print(f"Result: {self.string_rep()}")


    """

    CFOP method 1st phase: cross

    """
    
    def CFOP_cross(self) -> list[str]:
        """
        1st phase of CFOP method: create a cross on the bottom face.
        """

        moves = []

        # A restricted pool of possible algorithms
        pool = [key for key in self.all_moves.keys() if key[0] in ['F', 'B', 'U', 'D', 'L', 'R']]
        pool = pool + [x + " " + y for x in pool for y in pool] + [x + " " + y + " " + z for x in pool for y in pool for z in pool]

        # Try to build a cross using algorithms only from the pool
        # At least 2 arms of the cross (out of 4) will be solved in this loop 
        while True:
            best_stage = self.max_cross_value()

            for alg in pool:
                temp = RubiksCube(self.string_rep())
                temp.algorithm(alg)
                if temp.max_cross_value() > best_stage:
                    self.algorithm(alg)
                    moves.append(alg)
                    break

            # Break loop if can't improve
            if self.max_cross_value() <= best_stage:
                break

        # Move the face of the cross to the bottom
        move_to_bottom = {4: "X2", 13:"Z'", 22:"X'", 31:"Z", 40:"X", 49: ""}
        for key, value in self.cross_stages().items():
            if value == self.max_cross_value():
                self.algorithm(move_to_bottom[key])
                moves.append(move_to_bottom[key])
                break

        # Solve the remaining arms of the cross (at least 2 arms are already solved at this stage)
        # We move the incorrect arm of the cross to position 46, then use predefined algorithms to move the appropriate cell there
        while self.max_cross_value() < 4:
            
            # Rotate cube so that the incorrect arm of the cross moves to position 46
            if not self.colors_match([[49, 46], [25, 22]]):
                pass
            elif not self.colors_match([[49, 48], [16, 13]]):
                self.algorithm("Y'")
                moves.append("Y'")
            elif not self.colors_match([[49, 50], [34, 31]]):
                self.algorithm("Y")
                moves.append("Y")
            elif not self.colors_match([[49, 52], [43, 40]]):
                self.algorithm("Y2")
                moves.append("Y2")

            pool = ["F", "F'", 'F2',
                    "U2 F2", "U' F2", "U F2", "L' F'", "R F",
                    "L F' L'", "D' L D", "D R' D'", "R' F R", "D R D'", "D' L' D", "L2 U' F2", "R2 U F2",
                    "D' L2 D F'", "F' D' L D", "F D' L D", "D R2 D' F", "R U R' F", "B R2 F R2",
                    "B2 U R' F R"]
            best_stage = self.max_cross_value()
            for alg in pool:
                temp = RubiksCube(self.string_rep())
                temp.algorithm(alg)
                if temp.max_cross_value() > best_stage:
                    self.algorithm(alg)
                    moves.append(alg)
                    break

        # Check if cross is solved (on the bottom face)
        assert self.colors_match([[46, 48, 49, 50, 52], [13, 16], [22, 25], [31, 34], [40, 43]]), f"Something went wrong :("

        return moves

    def cross_stages(self) -> dict:
        """
        Return the stage of the cross on each face as a dict. Stage is the number of solved arms of the cross.
        Faces are represented by their middle positions.
        """

        middle_cells = [4, 13, 22, 31, 40, 49]
        cross_stages = {key: 0 for key in middle_cells}
        for arm in self.cross_arms:
            if self.state[arm[0]] == self.state[arm[1]] and self.state[arm[2]] == self.state[arm[3]]:
                cross_stages[arm[0]]  += 1
                cross_stages[arm[-1]] += 1
        return cross_stages
    
    def max_cross_value(self):
        """
        Return the maximum of cross stages.
        """
        return max(self.cross_stages().values())
    
    cross_arms = [[4, 1, 37, 40], [4, 3, 10, 13], [4, 5, 28, 31], [4, 7, 19, 22],
        [13, 14, 21, 22], [22, 23, 30, 31], [31, 32, 39, 40], [40, 41, 12, 13],
        [49, 46, 25, 22], [49, 48, 16, 13], [49, 50, 34, 31], [49, 52, 43, 40]]
   
    
    """

    CFOP method 2nd phase: F2L
    
    """

    def CFOP_F2L(self) -> list[str]:
        """
        2nd phase of CFOP method: solve the first two layers.
        """

        moves = []

        # F2L is done by pairing up matching corners and edges, and inserting these pairs into the correct slots using predefined algorithms.
        obscure_cases = 0  # since the list of algorithms above doesn't cover every possible case, we need to track how many obscure cases we find
        solved_pairs = [False, False, False, False]  # we track whether each pair has been solved
        i = 0  # this variable tracks which face of the cube we are currently on
        while not all(solved_pairs):  # we solve pairs in succession until all four are solved
            pattern = self.CFOP_F2L_pattern()  # we detect the pattern
            pattern_found = False
            if self.F2L_solved[0] in pattern[0][0] and self.F2L_solved[1] in pattern[0][1] and self.F2L_solved[2] in pattern[0][1] and self.F2L_solved[3] in pattern[0][2] and self.F2L_solved[4] in pattern[0][2]:  # first we check if the pair was already solved accidentally
                solved_pairs[i] = True
                i = (i+1) % 4
                self.move('Y') # move to the next face
                moves.append('Y')
                continue
            for j in range(4):  # we check against all 4 possible rotations of the pattern
                for k in range(77):  
                    if self.F2L_states[k][0] in pattern[j][0] and self.F2L_states[k][1] in pattern[j][1] and self.F2L_states[k][2] in pattern[j][1] and self.F2L_states[k][3] in pattern[j][2] and self.F2L_states[k][4] in pattern[j][2]:
                        # if the pattern matches a possible case, we execute the corresponding algorithm and set the cornen as solved
                        obscure_cases = 0  # executing an algorithm affects the rest of the cube, which has the potential to get us better cases
                        solved_pairs[i] = True
                        i = (i+1) % 4
                        pattern_found = True
                        self.algorithm('U', j)  # before executing the algorithm, we rotate the upper layer to match the correct pattern
                        self.algorithm(self.F2L_algorithms[k])
                        self.move('Y')  # move to the next face
                        moves.extend(['U'] * j)
                        moves.append(self.F2L_algorithms[k])
                        moves.append('Y')
                        break
                if pattern_found == True:
                    break
            if pattern_found == False: # if the pattern doesn't match any of the stored cases, we move on to the next pair (this can't happen on the final pair)
                i = (i+1) % 4
                obscure_cases += 1
                if obscure_cases == 4: # if all four corner-edge pairs are obscure cases, we need to make some arbitrary moves to get different cases (should be fairly rare)
                    obscure_cases = 0
                    self.algorithm("R U R'")
                    moves.append("R U R'")
                self.move('Y')
                moves.append('Y')

        # Check if F2L is solved
        assert self.colors_match([
            [12 + i for i in range(6)],
            [21 + i for i in range(6)],
            [30 + i for i in range(6)],
            [39 + i for i in range(6)],
            [45 + i for i in range(9)]
        ]), f"Something went wrong :("

        return moves

    def CFOP_F2L_pattern(self) -> list[list]:
        '''
        Find pattern of positions of bottom, front and right colors in the CFOP method's F2L phase.
        Return all 4 different patterns from rotating the upper layer.
        '''
        bottom_color = self.state[49]  # color of middle cell on the bottom face
        front_color = self.state[22]   # color of middle cell on the front face
        right_color = self.state[31]   # color of middle cell on the right face
        pattern = [[[],[],[]],[[],[],[]],[[],[],[]],[[],[],[]]]
        for i in range(4):
            for j in [0,1,3,5,7,8,9,10,12,14,15,17,19,20,21,23,24,26,27,28,30,32,33,35,37,38,39,41,42,44,45,47,51,53]:  # relevant positions
                if self.state[j] == bottom_color:
                    pattern[i][0].append(j)
                elif self.state[j] == front_color:
                    pattern[i][1].append(j)
                elif self.state[j] == right_color:
                    pattern[i][2].append(j)
            self.move("U")  # rotate upper layer to get different cases

        return pattern
    
    F2L_solved = [47,23,26,30,33]
    
    F2L_states = [[8,3,27,10,20], [8,27,37,1,20], [8,1,27,20,37], [8,10,27,3,20], [8,7,27,19,20], [8,27,28,5,20], [8,5,27,20,28], [8,19,27,7,20], #8
        [27,1,20,8,37], [20,8,10,3,27], [27,3,20,8,10], [20,8,37,1,27], [27,7,20,8,19], [20,8,28,5,27], [27,5,20,8,28], [20,8,19,7,27], #16
        [27,19,20,7,8], [20,5,8,27,28], [27,20,37,1,8], [20,3,8,10,27], [20,1,8,27,37], [27,10,20,3,8], [20,7,8,19,27], [27,20,28,5,8], #24
        [27,20,23,8,30], [20,8,23,27,30], [27,20,30,8,23], [20,8,30,23,27], [8,23,27,20,30], [8,27,30,20,23], [33,5,47,26,28], [26,19,33,7,47], #32
        [26,5,33,28,47], [33,19,47,7,26], [47,5,26,28,33], [47,19,26,7,33], [33,23,47,26,30], [26,23,33,30,47], [33,30,47,23,26], [26,30,33,23,47], #40
        [47,26,30,23,33], [8,27,39,20,32], [8,21,27,14,20], [8,27,32,20,39], [8,14,27,20,21], [0,9,41,12,38], [0,9,12,38,41], [27,20,39,8,32], #48
        [20,8,21,27,14], [27,20,32,8,39], [20,8,14,21,27], [27,20,21,8,14], [20,8,39,27,32], [27,14,20,8,21], [20,8,32,27,39], [9,38,41,0,12], #56
        [38,0,41,9,12], [9,12,38,0,41], [38,0,12,9,41], [53,28,35,5,42], [53,5,35,28,42], [35,28,42,5,53], [35,5,42,28,53], [42,28,53,5,35], #64
        [42,5,53,28,35], [45,7,17,19,24], [45,17,19,7,24], [24,7,45,17,19], [24,19,45,7,17], [17,7,24,19,45], [17,19,24,7,45], [51,3,44,10,15], #72
        [51,37,44,1,15], [44,3,15,10,51], [15,37,51,1,44], [15,3,51,10,44], [44,15,37,1,51]]

    F2L_algorithms = ["U2 R U R' U R U' R'", "F' L' U2 L F", "U R U2 R' U R U' R'", "U' F' U2 F U' F' U F", "U F R' F' R U R U R'", #5
        "F U R U' R' F' R U' R'", "R U2 R' U' R U R'", "F' U2 F U F' U' F", "R U R'", "F' U' F", "U' R U R' U R U R'", "U F' U' F U' F' U' F", #12
        "R' U2 R2 U R2 U R", "F U2 F2 U' F2 U' F'", "U' R U' R' U R U R'", "U F' U F U' F' U' F", "U' F' U F", "U R U' R'", "R' M U2 R2 U R2 U R M'", #19
        "U' R U2 R' U2 R U' R'", "U' R U R' U2 R U' R'", "U' R M' U' R' U R U R' M", "F' U F U2 R U R'", "R U' R' U2 F' U' F", "U R U R' U2 R U R'", #25
        "U' R U' R' U2 R U' R'", "U F' U' F U' R U R'", "U2 R U R' F R' F' R", "U R U' R' U R U' R' U R U' R'", "U' R' F R F' R U' R'", #30
        "R U R' U' R U R'", "R' F R F' R' F R F'", "R U' R' U R U' R'", "F' U F U' F' U F", "U' R' F R F' R U R'", "U R U' R' F R' F' R", #36
        "R U' R' U R U2 R' U R U' R'", "R U' R' U' R U R' U2 R U' R'", "R U' R' F' L' U2 L F", "F' L' U2 L F R U R'", "R2 U2 F R2 F' U2 R' U R'", #41
        "U' R' U R2 U' R'", "U F U' F2 U F", "U2 R' U R U' S R S'", "L F' L2 U L U2 F", "U2 R2 U E' R2 U' E R2", "R U E' R' U R U' E R'", "R' U' R2 U R'", #48
        "F U F2 U' F", "F D R D' F'", "L' F' U' F L", "U' L' U' L R U' R'", "Y U R U R' L' U L Y'", "F U2 F' R U R'", "R' U2 R F' U' F", #55
        "Y2 U R U R' L U L' Y2", "Y2 U' R U' R' L U' L' Y2", "Y2 U2 F' L U L' F Y2", "Y' U2 F R' U' R F' Y", "Y U R U' R' L' U L Y'", #60
        "Y2 L' U2 L U' L U L' Y2", "Y2 F' L F L' U' L U2 L' Y2", "Y2 U' L' U' L2 U2 L' Y2", "Y2 S' L S Y2", "Y2 U L' U L U L U L' Y2", #65
        "U' L' U L R U' R'", "Y' R U2 R' U R' U' R Y", "Y' F R' F' R U R' U2 R Y", "Y' U R U R2 U2 R Y", "Y' S R' S' Y", "Y' U' R U' R' U' R' U' R Y", #71
        "Y2 U' F' U F L U2 L' Y2", "Y2 U R U' R' U F' S' L F S Y2", "Y2 R U' R' L U2 L' Y2", "Y2 R U R' F' S' L F S Y2", "Y' L F' L' F R' U2 R Y", #76
        "Y2 R' F R F' L U2 L' Y2"]
                      

    """

    CFOP method 3rd phase: OLL
    
    """

    def CFOP_OLL(self) -> list[str]:
        '''
        3rd phase of CFOP method: orienting the last layer.
        '''
        
        moves = []

        # Find and apply appropriate algorithm for the current pattern
        pattern = self.CFOP_OLL_pattern() # the 4 rotationally symmetric forms of the pattern
        for i in range(4):
            if pattern[i] in self.OLL_states:
                idx = self.OLL_states.index(pattern[i])
                self.algorithm("Y", i)  # before applying the algorithm, the cube must be rotated to match with the ith form of the pattern
                self.algorithm(self.OLL_algorithms[idx])
                moves.extend(["Y"] * i)
                moves.append(self.OLL_algorithms[idx])
                break
            
        # Check if OLL is solved
        assert self.colors_match([
            [ 0 + i for i in range(9)],
            [12 + i for i in range(6)],
            [21 + i for i in range(6)],
            [30 + i for i in range(6)],
            [39 + i for i in range(6)],
            [45 + i for i in range(9)]
        ]), f"Something went wrong :("

        return moves

    def CFOP_OLL_pattern(self) -> list[list]:
        '''
        Find pattern of non-middle positions of the last color in CFOP method's OLL phase.
        I.e. positions on the last layer where the color is same as the middle of the last face.
        Return all 4 rotationally symmetric form of the pattern.
        '''

        last_color = self.state[4]  # color of middle cell on the last face (which is the face 'up')
        pattern = [[],[],[],[]]
        for i in range(4):
            for j in [0,1,2,3,5,6,7,8,9,10,11,18,19,20,27,28,29,36,37,38]:  # non-middle positions on the last layer
                if self.state[j] == last_color:
                    pattern[i].append(j)
            self.move("Y")  # rotate cube to get rotationally symmetric cases

        return pattern

    OLL_states = [[9,10,11,19,27,28,29,37], [9,10,11,19,20,28,36,37], [8,10,11,19,28,29,37,38], [2,9,10,18,19,27,28,37], [5,7,8,10,11,29,37,38], #5
        [1,2,5,9,10,18,19,27], [1,3,6,19,20,28,29,38], [0,3,7,18,27,28,36,37], [1,3,8,9,18,19,28,36], [2,3,7,11,20,28,37,38], #10
        [5,6,7,10,20,29,37,38], [3,7,8,9,18,28,36,37], [3,5,6,19,20,29,37,38], [3,5,8,9,18,19,36,37], [3,5,8,11,19,29,37,38], #15
        [2,3,5,9,18,19,27,37], [0,8,10,11,19,28,36,37], [0,2,10,18,19,20,28,37], [0,2,10,11,19,27,28,37], [0,2,6,8,10,19,28,37], #20
        [1,3,5,7,18,20,36,38], [1,3,5,7,9,11,20,36], [0,1,2,3,5,7,18,20], [1,2,3,5,7,8,18,38], [1,2,3,5,6,7,9,20], #25
        [1,2,3,5,7,9,18,27], [1,3,5,6,7,20,29,38], [0,1,2,3,6,8,19,28], [0,2,3,7,11,27,28,37], [1,3,6,8,9,19,28,29], #30
        [1,2,5,8,10,18,19,38], [2,5,7,8,10,18,37,38], [2,3,5,8,18,19,37,38], [3,5,6,8,9,19,29,37], [0,5,7,8,10,18,29,37], #35
        [0,3,7,8,18,28,29,37], [0,1,3,8,18,19,28,29], [1,2,3,6,19,27,28,38], [2,3,5,6,19,27,37,38], [0,3,5,8,11,19,36,37], #40
        [1,3,6,8,19,28,36,38], [0,2,3,7,18,20,28,37], [0,1,2,3,18,19,20,28], [2,5,7,8,9,10,11,37], [2,3,5,8,9,11,19,37], #45
        [0,1,6,7,10,27,28,29], [1,5,10,18,19,27,29,38], [1,3,9,11,19,20,28,36], [1,5,9,10,11,19,20,36], [5,7,9,10,11,20,36,37], #50
        [3,5,9,11,19,20,36,37], [1,7,10,18,27,28,29,38], [5,7,9,10,11,27,29,37], [1,5,9,10,11,19,27,29], [1,7,9,10,11,27,28,29], #55
        [3,5,9,11,19,27,29,37], [0,2,3,5,6,8,19,37]]
    
    OLL_algorithms = ["R U2 R2 F R F' U2 R' F R F'", "F R U R' U' F' F S R U R' U' F' S'", "F S R U R' U' F' S' U' F R U R' U' F'", #3
        "F S R U R' U' F' S' U F R U R' U' F'", "R' M U2 R U R' U R M'", "R M' U2 R' U' R U' R' M", #6
        "R M' U R' U R U2 R' M", "R' M U' R U' R' U2 R M'", "R U R' U' R' F R2 U R' U' F'", #9
        "R U R' U R' F R F' R U2 R'", "R' M R2 U R' U R U2 R' U M'", "Y2 M' R' U' R U' R' U2 R U' M", #12
        "R M' U' R' M U' R M' U R' M Y' R' U R", "R' F R U R' F' R F U' F'", "R' M U' R M' R' U' R U R' M U R M'", #15
        "R M' U R' M R U R' U' R M' U' R' M", "R U R' U R' F R F' U2 R' F R F'", "R M' U R' U R U2 R2 M2 U' R U' R' U2 R M'", #18
        "M U R U R' U' M' R' F R F'", "M U R U R' U' M2 U R U' R' M", "R U2 R' U' R U R' U' R U' R'", #21
        "R U2 R2 U' R2 U' R2 U2 R", "R2 D R' U2 R D' R' U2 R'", "R M' U R' U' R' M F R F'", #24
        "F' R M' U R' U' R' M F R", "R U2 R' U' R U' R'", "R U R' U R U2 R'", #27
        "R M' U R' U' M U R U' R'", "Y R U R' U' R U' R' F' U' F R U R'", "F U R U2 R' U' R U2 R' U' F'", #30
        "R' U' F U R U' R' F' R", "R U B' U' R' U R B R'", "R U R' U' R' F R F'", #33
        "R U R2 U' R' F R U R U' F'", "R U2 R2 F R F' R U2 R'", "R' U' R U' R' U R U L M U' R' U X", #36
        "F R U' R' U' R U R' F'", "R U R' U R U' R' U' R' F R F'", "L F' L' U' L U F U' L'", #39
        "R' F R U R' U' F' U R", "R U R' U R U2 R' F R U R' U' F'", "R' U' R U' R' U2 R F R U R' U' F'", #42
        "R' U' F' U F R", "F S R U R' U' F' S'", "F R U R' U' F'", #45
        "R' U' R' F R F' U R", "F' L' U' L U L' U' L U F", "F R U R' U' R U R' U' F'", #48
        "R M' U' R2 M2 U R2 M2 U R2 M2 U' R M'", "R' M U R2 M2 U' R2 M2 U' R2 M2 U R' M", "F S R U R' U' R U R' U' F' S'", #51
        "R' U' R U' R' U Y' R' U R B", "R' M U' R U' R' U R U' R' U2 R M'", "R M' U R' U R U' R' U R U2 R' M", #54
        "Y R' F R U R U' R2 F' R2 U' R' U R U R'", "R' M U' R M' U' R' U R U' R' U R R' M U R M'", "R U R' U' M' U R U' R' M"] #57  


    """

    CFOP method 4th phase: PLL
    
    """
    
    def CFOP_PLL(self) -> list[str]:
        '''
        4th phase of CFOP method: permutate the last layer.
        '''
        
        moves = []

        # Find and apply appropriate algorithm for the current pattern
        pattern = self.CFOP_PLL_pattern() # the 4 rotationally symmetric forms of the pattern
        for i in range(4):
            if pattern[i] in self.PLL_states:
                idx = self.PLL_states.index(pattern[i])
                self.algorithm("Y", i)  # before applying the algorithm, the cube must be rotated to match with the ith form of the pattern
                self.algorithm(self.PLL_algorithms[idx])
                moves.extend(["Y"] * i)
                moves.append(self.PLL_algorithms[idx])
                break

        # In some cases, we may need to take one more move to complete the solution
        if self.state[10] == self.state[22]:
            self.move("U'")
            moves.append("U'")
        elif self.state[10] == self.state[31]:
            self.move("U2")
            moves.append("U2")
        elif self.state[10] == self.state[40]:
            self.move("U")
            moves.append("U")

        # Check if PLL is solved
        assert self.colors_match([
            [ 0 + i for i in range(9)],
            [ 9 + i for i in range(9)],
            [18 + i for i in range(9)],
            [27 + i for i in range(9)],
            [36 + i for i in range(9)],
            [45 + i for i in range(9)]
        ]), f"Something went wrong :("

        return moves

    def CFOP_PLL_pattern(self) -> list[list]:
        '''
        Find pattern of different colors on the side of the last layer in CFOP method's PLL phase.
        A pattern is a list of size 12, where positions with indices i, i+1 and i+2 (i in {0, 3, 6, 9}) have the same color.
        Return all 4 rotationally symmetric form of the color-neutral pattern.
        '''

        # Find pattern of different colors on the side of the last layer
        colors = [self.state[13], self.state[22], self.state[31], self.state[40]] # colors of faces 'left', 'front', 'right', 'back'
        pattern = [[],[],[],[]]
        for i in range(4):
            for c in colors:
                for j in [9,10,11,18,19,20,27,28,29,36,37,38]:  # positions on the side of the last layer
                    if self.state[j] == c:
                        pattern[i].append(j)
            self.move("Y")  # rotate cube to get rotationally symmetric cases
                 
        # Make each form of the pattern color-neutral
        # We achieve this by starting each list with position '9', which is the top left cell of face 'left'
        for i in range(4):
            j = pattern[i].index(9)
            pattern[i] = pattern[i][j:12] + pattern[i][0:j]

        return pattern
    
    PLL_states = [[9,11,28,10,18,20,19,27,29,36,37,38], [9,11,19,18,20,28,10,27,29,36,37,38], [9,11,19,10,18,20,27,29,37,28,36,38],
        [9,11,28,18,20,37,10,27,29,19,36,38], [9,20,37,10,11,27,18,19,29,28,36,38], [9,20,28,27,29,37,10,11,36,18,19,38],
        [9,19,29,20,28,36,11,27,37,10,18,38], [9,11,37,18,19,29,20,28,36,10,27,38], [9,29,37,10,11,36,18,20,28,19,27,38],
        [9,10,29,11,36,37,18,19,20,27,28,38], [9,10,11,18,28,29,19,20,36,27,37,38], [9,11,28,18,19,29,10,20,36,27,37,38],
        [9,10,11,18,29,37,20,28,36,19,27,38], [9,29,37,20,28,36,10,11,27,18,19,38], [9,28,29,10,20,36,11,27,37,18,19,38],
        [9,28,29,19,20,36,10,11,27,18,37,38], [9,10,29,20,36,37,11,27,28,18,19,38], [9,11,28,18,29,37,19,20,36,10,27,38],
        [9,19,20,11,27,37,10,18,29,28,36,38], [9,11,28,10,18,29,20,36,37,19,27,38], [9,11,19,18,29,37,10,20,36,27,28,38]]
    
    PLL_algorithms = ["R U' R U R U R U' R' U' R2", "R2 U R U R' U' R' U' R' U R'", "M2 U' M2 U' M' U2 M2 U2 M' U2",
        "M2 U' M2 U2 M2 U' M2", "X R' U R' D2 R U' R' D2 X' R2", "X R2 D2 R U R' D2 R U' X' R",
        "X' R U' R' D R U R' D' R U R' D R U' R' D' X", "R U' R' U' R U R D R' U' R D' R' U2 R' U'", "R' U2 R U2 R' F R U R' U' R' F' R2 U'",
        "R' U L' U2 R U' R' U2 R L U'", "R U R' F' R U R' U' R' F R2 U' R' U'", "R U R' U' R' F R2 U' R' U' R U R' F'",
        "R' U' F' R U R' U' R' F R2 U' R' U' R U R' U R", "R' U R' U' Y R' F' R2 U' R' U R' F R F", "F R U' R' U' R U R' F' R U R' U' R' F R F'",
        "R U R' U R U R' F' R U R' U' R' F R2 U' R' U2 R U' R'", "R' U R U' R' F' U' F R U R' F R' F' R U' R", "R2 U E' R' U R' U' R U' E R2 Y' R' U R",
        "F' U' F R2 U E' R' U R U' R U' E R2", "R2 U' R U' R U R' U R2 D' U R U' R' D U'", "R U R' Y' R2 U' E R U' R' U R' U E' R2"]

In [ ]:
cube = RubiksCube()
cube.show()

In [ ]:
cube.move("F")
cube.show()

In [ ]:
cube.reset()
cube.algorithm("U R2 F B R B2 R U2 L B2 R U' D' R2 F R' L B2 U2 F2")
cube.show()

In [ ]:
random_cube = RubiksCube()
random_cube.scramble()
random_cube.show()

In [ ]:
random_cube.solve()
random_cube.show()

In [ ]:
test_cube = RubiksCube("wbobrgbyygyrobryrbyrowwgrwgbywwgoyrgboogywworwbogobgyr")
test_cube.show()
test_cube.solve()
test_cube.show()